# Data Preperation

In [1]:
from scipy import ndimage
import numpy as np
np.random.seed(123)

In [2]:
import pandas as pd

In [3]:
# tmp_array = ndimage.imread('F:/cifar10/data/train/2.png')

# tmp_array.shape

In [4]:
def convert_images_to_array(path, end, start = 0):
    lst_train = []
#     train_path = 'F:/cifar10/data/train/'
    for i in range(start, end):
        val = (str(i+1))
    #     print(train_path + val+'.png')
        lst_train.append(ndimage.imread(path + val+'.png'))

    #Convert list to nd array    
    return(np.array(lst_train))

In [ ]:
# convert train data to ndarray
train_path = 'F:/cifar10/data/train/'
np_train = convert_images_to_array(train_path, 50000)

In [ ]:
print(np_train.shape)

In [ ]:
# Normalize data
np_train = np_train.astype('float32')
np_train = np_train/255.0

## creating labels

In [5]:
df_label = pd.read_csv('F:/cifar10/data/trainLabels.csv')

In [6]:
df_label.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [7]:
## Create a dictionary with unique labels assigned number
lab = list(df_label['label'].unique())
lab_dic = {}
for i in range(len(lab)):
#     print(i)
#     print(lab[i])
    lab_dic[lab[i]] = i
lab_dic

{'airplane': 9,
 'automobile': 3,
 'bird': 4,
 'cat': 7,
 'deer': 2,
 'dog': 8,
 'frog': 0,
 'horse': 5,
 'ship': 6,
 'truck': 1}

In [ ]:
a = 'cat'
lab_dic.get(a)

In [ ]:
# [lab_dic.get(i) for i in df_label['label'] ]

In [ ]:
# df_label['label'].unique()

In [ ]:
np_label = np.array([lab_dic.get(i) for i in df_label['label'] ])

In [ ]:
np_label.shape

In [ ]:
import keras

In [ ]:
y_nplabel = keras.utils.to_categorical(np_label)

In [ ]:
num_classes = y_nplabel.shape[1]
num_classes

# Deploy CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [ ]:
epochs = 50

In [ ]:
# # Create the model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))
# # Compile model
# epochs = 25
# lrate = 0.01
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# print(model.summary())

In [ ]:
def simplex_model():
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(32, (3,3), input_shape = (32,32,3), activation='relu'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(1, 1)))

    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

#     epochs = 20
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    
    return model

In [ ]:
def simple_model():
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(32, (3,3), input_shape = (32,32,3), activation='relu'))

    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))


    
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    
    return model


In [ ]:
model = simple_model()

In [ ]:
# Fit the model
model.fit(np_train, y_nplabel, validation_split=0.3, epochs=epochs, batch_size=32)

In [ ]:
model.save('F:/cifar10/model/my_simple_model_50.h5')

## Prediction

In [8]:
## load save model
from keras.models import load_model

Using TensorFlow backend.


In [9]:
my_model = load_model('F:/cifar10/model/my_simple_model_50.h5')

In [20]:
def get_batch_submission(model, start=0, end=300000, batch_size=1000):
    # convert test data to ndarray
    test_path = 'F:/cifar10/data/test/'

    lst_prd = []
    tmp_lst = []

    for i in np.arange(start, end, batch_size):
        if(i + batch_size > end):
            if(i + batch_size < end):
                pass
            else:
                tmp_lst.append(tuple((i, end)))
        else:
            tmp_lst.append(tuple((i, i + batch_size)))

    for i, j in tmp_lst:
        np_test = convert_images_to_array(test_path, start=i, end=j)
        print('start: ', i, 'end: ', j)
        print('batch size: ', np_test.shape)
        # Normalize data
        np_test = np_test.astype('float32')
        np_test = np_test/255.0

        np_prediction = model.predict_classes(np_test)

        for j in np_prediction:
            for i in lab_dic.items():
                if(i[1] == j):
                    lst_prd.append(i[0])
    print('Batch Completed')

    df_submission = pd.DataFrame()
    df_submission['id'] = list(range(1,len(lst_prd)+1))
    df_submission['label'] = lst_prd

    return df_submission

In [ ]:
df_submission = get_batch_submission(my_model, batch_size= 5000)

In [ ]:
df_submission.to_csv('F:/cifar10/data/submission.csv', index=False)

In [ ]:
def get_submission(model, data):
    np_prediction = model.predict_classes(data)

    lst_prd = []
    for j in np_prediction:
        for i in lab_dic.items():
            if(i[1] == j):
                lst_prd.append(i[0])
    #             print(i)
    #             print(i[1])
    #             print(i[0])
    lst_prd


    df_submission = pd.DataFrame()
    df_submission['id'] = list(range(1,len(np_prediction)+1))
    df_submission['label'] = lst_prd

    return df_submission

In [ ]:
# df_submission = get_submission(my_model,np_test)

### Visualize the data

In [ ]:
from matplotlib import pyplot
from scipy.misc import toimage

In [ ]:
pyplot.imshow(toimage(X_train[3]))
pyplot.show()

In [ ]:

# start = 0
# end = 17
# batch_size=3

# # convert test data to ndarray
# test_path = 'F:/cifar10/data/test/'
# # np_test = convert_images_to_array(test_path, 300000)

# lst_prd = []
# tmp_lst = []

# # for i in xrange(0, len(worklist), batch_size):
# for i in np.arange(start, end, batch_size):
#     if(i + batch_size > end):
#         if(i + batch_size < end):
#             pass
#         else:
# #             print(i, end)
#             tmp_lst.append(tuple((i, end)))
#     else:
# #         print(i, i + batch_size)
#         tmp_lst.append(tuple((i, i + batch_size)))

# for i, j in tmp_lst:
#     np_test = convert_images_to_array(test_path, start=i, end=j)
   
#     # Normalize data
#     np_test = np_test.astype('float32')
#     np_test = np_test/255.0
    
#     print('test data shape', np_test.shape)
    
#     np_prediction = model.predict_classes(np_test)

    
#     for j in np_prediction:
#         for i in lab_dic.items():
#             if(i[1] == j):
#                 lst_prd.append(i[0])
#     #             print(i)
#     #             print(i[1])
#     #             print(i[0])
#     lst_prd
    
# print(lst_prd)
# df_submission = pd.DataFrame()
# df_submission['id'] = list(range(1,len(np_prediction)+1))
# df_submission['label'] = lst_prd

# df_submission

#     #     print(i)
#     #     print(j)